In [1]:
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import urllib.request
from pathlib import Path
import os
import re
from time import sleep
import requests
from lxml import html
from urllib.request import urlopen
from urllib.error import HTTPError
from creds import USERNAME, PASSWORD, COURSE_TITLE, PS_LIB_URL, PS_ID_URL

In [2]:
driver = Firefox()
driver.get(PS_LIB_URL)

In [ ]:
# driver.find_element_by_id('Username').send_keys(USERNAME)
# driver.find_element_by_id('Password').send_keys(PASSWORD)
# driver.find_element_by_id('login').click()

In [3]:
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.LINK_TEXT, COURSE_TITLE)))
main = driver.find_element_by_link_text(COURSE_TITLE)
driver.get(main.get_attribute("href"))

In [4]:
driver.find_element_by_class_name('course-hero__button').click()
# driver.find_element_by_xpath("//button[.='Expand All']").click()

In [ ]:
sleep(10) #to manually expand all modules
soup = BeautifulSoup(driver.page_source, 'lxml')
modules = soup.find_all(class_="module")
module_content = driver.find_elements_by_class_name("module")
try:
    
    Path(COURSE_TITLE).mkdir(parents=True, exist_ok=True)
    os.chdir(COURSE_TITLE)

    for btns_clickable_list,lorem in zip(module_content,modules):
        module_header = lorem.find("h2")
        Path(module_header.text).mkdir(parents=True, exist_ok=True)
        os.chdir(module_header.text)
        btns_clickable = btns_clickable_list.find_elements_by_class_name("content-item")
        for btn in btns_clickable:
            btn.click()
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, "video-element")))
            soup = BeautifulSoup(driver.page_source, 'lxml')
            video_element = soup.find(id="video-element")
            urllib.request.urlretrieve(video_element.get('src'), btn.find_element_by_tag_name("h3").text)
            print(module_header.text + '\n' + btn.find_element_by_tag_name("h3").text + '\n' + video_element.get('src'))
        os.chdir('..')
    os.chdir('..')
    
except TimeoutError:
    print("TimeoutError")
    os.chdir('')
finally:
    driver.quit()

In [6]:
driver.quit()

In [ ]:
soup = BeautifulSoup(driver.page_source,'lxml')
drawers = soup.find_all(class_= re.compile('^drawer*'))
Path(COURSE_TITLE).mkdir(parents=True, exist_ok=True)
os.chdir(COURSE_TITLE)

for module in drawers:
    print(module.find('h3').text) #module-header
    Path(module.find('h3').text).mkdir(parents=True, exist_ok=True)
    os.chdir(module.find('h3').text)
    lorem = module.find_all('a', class_= re.compile('^clipListTitle*'))
    for ipsum in lorem:
        print(ipsum.text) #video-title
        print('https://app.pluralsight.com'+ipsum['href']) #video-player-link
#         lorem = requests.get('https://app.pluralsight.com'+ipsum['href'])
#         print(lorem.text)
    
#         soup = BeautifulSoup(lorem.text, 'lxml')
#         WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "video-element")))
#         video_element = soup.find(id="video-element")
#         print(video_element.get('src'))
#         urllib.request.urlretrieve(video_element.get('src'), ipsum.text)
        urllib.request.urlretrieve('http://techslides.com/demos/sample-videos/small.mp4', ipsum.text)
    print('\n')
    os.chdir('..')
os.chdir('..')

In [ ]:
session_obj = requests.session()
session_obj.headers.update({ 'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:68.0) Gecko/20100101 Firefox/68.0'})

response_obj = session_obj.get(PS_LIB_URL)

In [ ]:
# print(response_obj.text)
# print(session_obj.cookies)
soup = BeautifulSoup(response_obj.content, 'html.parser')
csrf_token = soup.find(attrs={"name":"__RequestVerificationToken"}).get('value')
showcaptcha = soup.find(id='ShowCaptcha').get('value')
recaptcha = soup.find(id='ReCaptchaSiteKey').get('value')
# print(type(session_obj.cookies))
# csrf_token = session_obj.cookies['Cookie __RequestVerificationToken_L2lk0']
# print(csrf_token)

In [ ]:
payload = {
    "__RequestVerificationToken" : csrf_token,
    "RedirectUrl" : "",
    "Username" : USERNAME,
    "Password" : PASSWORD,
    "ShowCaptcha" : showcaptcha,
    "ReCaptchaSiteKey" : recaptcha
    
}
print(payload)

In [ ]:


# session_obj.headers.update({ 'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:68.0) Gecko/20100101 Firefox/68.0'})
post = session_obj.post(pslib, data=payload, headers=dict(Referer=psid))
# print(post.headers)
# soup2 = BeautifulSoup(post.text, 'html.parser')
# print(soup2.prettify())

In [ ]:
get = session_obj.get(pslib)
soup1 = BeautifulSoup(get.content, 'html.parser')
print(soup1.prettify())

In [ ]:
session_obj = requests.Session()
session_obj.headers.update({ 'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:68.0) Gecko/20100101 Firefox/68.0'})

post = session_obj.post(PS_ID_URL, data = payload)
soup2 = BeautifulSoup(post.text, 'lxml')
print(soup2.prettify())

In [ ]:
get = session_obj.get(PS_LIB_URL)